<a href="https://colab.research.google.com/github/vivekswamy021/XGboost/blob/main/XGBoost_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Importing Libraries**

In [65]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

#**Loading  the data**

In [66]:
from google.colab import files
uploaded = files.upload()

Saving LoanData.csv to LoanData (2).csv


In [67]:
df=pd.read_csv("LoanData.csv")
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


#**EXtracting numerical_columns and categorical_columns from the data**

In [68]:
numerical_cols =['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term']

categorical_cols=['Gender','Married','Dependents','Education','Self_Employed','Credit_History','Property_Area','Loan_Status']

#**Data Cleaning**

In [69]:
# Drop Loan_ID (not useful for prediction)
df.drop(columns=['Loan_ID'], inplace=True)

# selecting column name from categorical and numerical
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = df.select_dtypes(exclude=['object']).columns

#adding values
imputer_categorical = SimpleImputer(strategy='most_frequent') # categorical_col/ mode
imputer_numerical = SimpleImputer(strategy='median')        # for numerical_col / median

# fit and transform missing values
df[categorical_cols] = imputer_categorical.fit_transform(df[categorical_cols])
df[numerical_cols] = imputer_numerical.fit_transform(df[numerical_cols])

# Encode categorical variables
encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])

#Split Data into Features (X) & Target (y)
X = df.drop(columns=['Loan_Status'])
y = df['Loan_Status']

# Split Data into Training & Testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [60]:
# Initialize and train XGBoost classifier
model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    use_label_encoder=False,
    n_estimators=50,  # Boosting rounds
    max_depth=2,       # Limit tree depth
    learning_rate=0.2  #  learning rate
)

model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(accuracy)


0.7886178861788617


#**CV score(cross validation score)**

In [61]:
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")

cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# Print CV scores and mean accuracy
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Mean CV Accuracy: {cv_scores.mean():.4f}")

Cross-Validation Scores: [0.86868687 0.80612245 0.78571429 0.78571429 0.79591837]
Mean CV Accuracy: 0.8084
